In [1]:
!pip install langchain-experimental langchain-community langchain networkx langchain-google-genai langchain-core json-repair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00


In [31]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import GoogleGenerativeAI
import time
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
import google.generativeai as genai
from google.colab import userdata

In [39]:
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

llm = GoogleGenerativeAI(model="gemini-1.5-flash",goolge_api_key=GOOGLE_API_KEY)

In [24]:
text = """ Marie Curie, née Maria Sklodowska, was born in Warsaw on November 7, 1867, the daughter of a secondary-school teacher. She received a general education in local schools and some scientific training from her father. She became involved in a students’ revolutionary organization and found it prudent to leave Warsaw, then in the part of Poland dominated by Russia, for Cracow, which at that time was under Austrian rule. In 1891, she went to Paris to continue her studies at the Sorbonne where she obtained Licenciateships in Physics and the Mathematical Sciences."""

In [40]:
genai.configure(api_key=GOOGLE_API_KEY)
documents = [Document(page_content=text)]
llm_transformers = LLMGraphTransformer(llm=llm)
graph_document = llm_transformers.convert_to_graph_documents(documents)

In [43]:
llm_transformers_filtered = LLMGraphTransformer(llm=llm, allowed_nodes=["Person","Country","Organization"],allowed_relationships=["NATIONALITY","LOCATED_IN","WORKED_AT","SPOUSE","MOTHER"])
graph_documents_filtered = llm_transformers_filtered.convert_to_graph_documents(documents)

In [51]:
graph = NetworkxEntityGraph()
for node in graph_documents_filtered[0].nodes:
  graph.add_node(node.id)


In [55]:
for edge in graph_documents_filtered[0].relationships:
  graph._graph.add_edge(edge.source.id,edge.target.id,relation=edge.type)

In [56]:
chain = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

In [57]:
question = """Tell me about Marie Curie"""
chain.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie

Full Context:
Marie Curie NATIONALITY Poland
Marie Curie LOCATED_IN Warsaw
Marie Curie LOCATED_IN Paris
Marie Curie WORKED_AT Sorbonne

> Finished chain.


'Marie Curie was Polish.  She was located in both Warsaw and Paris. She worked at the Sorbonne.\n'